# ラスターファイルの並列分散処理

このエクササイズでは、GDAL ライブラリを利用して複数の GeoTiff ラスターファイルを並列分散処理する方法を学びます。<br>
ファイル単位の処理を分散して行う場合、事前にファイルリストを作成し RDD の仕組みを使ってそのファイルリストを分散させて、各クラスター ノードごとに割り当てられたファイルを処理する手法を取ります。
以下の順に解説します。

1. Spark の RDD の仕組みを確認
1. Driver 上で GeoTIFF to PNG 変換処理（非分散）
1. Worker 上で GeoTIFF to PNG 変換処理（分散）

In [0]:
from pyspark import TaskContext
import glob
import os

## データ分散の仕組み確認

Apache SparkのResilient Distributed Dataset（RDD）は、サイズが大きすぎて1つのノードに収まらないため、複数のノードに分割する必要があるさまざまなデータの集合です。Apache Sparkは自動的にRDDをパーティションに分割し、複数のノードに分散します。この操作は遅延評価され（たとえば、アクションがトリガーされるまで実行を開始しないことで管理性が高まり、計算量が低減するため、結果的に最適化と速度が向上します）<br><br>

- https://www.talend.com/jp/resources/intro-apache-spark-partitioning/
- https://0x0fff.com/spark-architecture/

In [0]:
# parallelizing data collection
my_list = [1, 2, 3, 4, 5]
my_list_rdd = sc.parallelize(my_list)


In [0]:
type(my_list_rdd)

## パーティション分割の確認

分割されたパーティション数を確認し、パーティションに分割された状態をファイルに出力する。

Sparkで使用されるパーティションの数は設定が可能で、少なすぎると同時実行性の低下、データの偏り（データスキュー）、不適切なリソース利用の原因となり、多すぎるとタスクスケジューリングの所要時間が実際の実行時間より長くなるなどの問題が発生します。デフォルトでは、すべてのexecutorノード上のコアの総数に設定されています。Sparkはパーティションごとに1つのタスクを割り当て、各Workerは一度に1つのタスクを処理できます。

In [0]:
my_list_rdd.getNumPartitions()

In [0]:
my_list_rdd.saveAsTextFile("/mnt/testblob/outputText")

### データを1か所にまとめる
`collect()`は、データセットのすべての要素を (すべてのノードから) ドライバー ノードに取得するために使用されるアクション操作です。通常は`filter()`、`group()`などの後に小さなデータセットで `collect()` を使用する必要があります。大きなデータセットを取得するとOutOfMemoryエラーが発生します。

In [0]:
my_list_rdd.collect()

In [0]:
my_list_rdd.coalesce(1).saveAsTextFile("/mnt/testblob/outputTextMerged")

## Driver 上で GeoTIFF to PNG 変換処理（非分散）
Python は既定でシングルスレッドです。この例では、GeoTIFF ファイルを一つずつ処理します。約 4 分程度かかります。

In [0]:
!mkdir -p /dbfs/mnt/testblob/png/

In [0]:
from osgeo import gdal
import os

inputDir =  "/dbfs/mnt/testblob/geotiff/"
outputDir = "/dbfs/mnt/testblob/png/"

files = glob.glob(inputDir + "*.tif")
os.environ['GDAL_PAM_ENABLED'] = 'NO'

for f in files:
    gdal.Translate(outputDir + os.path.split(f)[1][:-4] + ".png", f, outputType=gdal.GDT_Byte)
    print(os.path.split(f)[1])


## Worker 上で GeoTIFF to PNG 変換処理（分散）

ファイルを並列に読み込むために、Sparkにファイル一覧の並列化を指示する必要があります。<br>
RDDの各データセットは  論理パーティションに分割され、クラスターの異なるノードで計算される場合があります。
パーティションは、PySpark における並列処理の基本単位です。
PySpark の RDD はパーティションのコレクションであることを思い出してください。

In [0]:
#1. 分散処理したいファイルの一覧を作成
inputDir =  "/dbfs/mnt/testblob/geotiff/"
files = glob.glob(inputDir + "*.tif")
#ファイル一覧は各クラスターに分散して保持されます
data_files = sc.parallelize(files).cache()

data_files.count()

In [0]:
data_files.take(3)

Sparkでは、Sparkクラスターの各ワーカーノード上でexecutor（JavaVM）が動き、その中で複数のタスクを（マルチスレッドで）並列に処理するので、スレッドセーフになるようにプログラミングする必要があります。

ただし、RDDのmapやfilter等に渡す関数（関数オブジェクト）はexecutorに渡す際にシリアライズされ、executorの各タスクで別々にデシリアライズされます。
つまりタスク毎に別インスタンスになるので、関数の中のインスタンスが共有されることはありません。
（例えば関数の外でインスタンス化したSimpleDateFormat（SimpleDateFormatはスレッドセーフではない）を関数の中で使ってもよい。むしろ複数タスク間で共有したい場合は共有変数を使う必要がある）

In [0]:
!mkdir -p /dbfs/mnt/testblob/png_rdd/

In [0]:
%%time
from osgeo import gdal, gdalconst, gdal_array
import os

def translateGeotiff2Png(file):
    try:
        outputDir = "/dbfs/mnt/testblob/png_rdd/"
        os.environ['GDAL_PAM_ENABLED'] = 'NO'
        src = gdal.Open(file, gdalconst.GA_ReadOnly)
        src = gdal.Translate(outputDir + os.path.split(file)[1][:-4] + ".png", file, outputType=gdal.GDT_Byte)
        
        tasks = []
        ctx = TaskContext()
        #処理しているファイル、タスク情報をセット
        tasks.append({'name': src.GetDescription(),
                      'x': src.RasterXSize,
                      'y': src.RasterYSize,
                      'rasterCount': src.RasterCount,
                      'stageId': ctx.stageId(),
                      'partitionId': ctx.partitionId(),
                      'taskAttemptId': ctx.taskAttemptId()
                      })
        
    except Exception as e:
        print('Error translateGeotiff2Png:' + file)
        print(e)
        
    return tasks


tasks = data_files.map(translateGeotiff2Png)
tasks.collect()